<a href="https://colab.research.google.com/github/m3llad0/TC3007C/blob/master/preprocesamiento_arima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from zipfile import ZipFile


In [12]:
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d bryanchungweather/nba-player-stats-dataset-for-the-2023-2024

with ZipFile('nba-player-stats-dataset-for-the-2023-2024.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

!rm nba-player-stats-dataset-for-the-2023-2024.zip

  0% 0.00/21.5k [00:00<?, ?B/s]
100% 21.5k/21.5k [00:00<00:00, 13.2MB/s]


In [3]:
# Cargar set de datos
df = pd.read_csv()